In [34]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
df = pd.read_excel('Data/pc6_2024_v1.xlsx')

In [31]:
df.loc[(df['Inwoner, huishouden en woninggegevens per postcode, peildatum 1 januari 2024']   == '5056TG'), 'Unnamed: 25']

245944    -99997
Name: Unnamed: 25, dtype: object

In [32]:
BASE_URL = "https://api.pdok.nl/cbs/postcode6/ogc/v1"

In [35]:
url = f"{BASE_URL}/collections"
resp = requests.get(url)

In [43]:
url = f"{BASE_URL}/collections/postcode6"
params = {
    "f": "json",
    "postcode6": '5056TG'  # Not official, but typical filter pattern
}
resp = requests.get(url, params=params)

In [46]:
data = resp.json()

In [54]:
data['extent']['spatial']['bbox'][0]

[-1.657292, 48.040502, 12.431727, 56.11059]

In [55]:
data['extent']['spatial']['crs']

'http://www.opengis.net/def/crs/OGC/1.3/CRS84'

In [45]:
import requests
import geopandas as gpd
from shapely.geometry import shape

BASE_URL = "https://api.pdok.nl/cbs/postcode6/ogc/v1"

def get_features_by_bbox(collection, bbox, crs="EPSG:4326", limit=100):
    url = f"{BASE_URL}/collections/{collection}/items"
    params = {
        "bbox": ",".join(map(str, bbox)),
        "crs": crs,
        "limit": limit,
        "f": "json"
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    
    # Convert to GeoDataFrame
    features = data.get("features", [])
    gdf = gpd.GeoDataFrame.from_features(features, crs=crs)
    return gdf

# Example: Amsterdam center
bbox = [4.8, 52.35, 4.9, 52.4]  # minX, minY, maxX, maxY (lon/lat)
collection = "postcode6"

gdf = get_features_by_bbox(collection, bbox, crs="EPSG:4326")
print(gdf.head())

# Plot with GeoPandas
gdf.plot(figsize=(10, 10))

HTTPError: 400 Client Error: Bad Request for url: https://api.pdok.nl/cbs/postcode6/ogc/v1/collections/postcode6/items?bbox=4.8%2C52.35%2C4.9%2C52.4&crs=EPSG%3A4326&limit=100&f=json

In [177]:
response = supabase.table(table_name).select("id, car_id, make, listing_price").execute()
car_ids_in_database = response.data

In [178]:
df_full = pd.DataFrame(car_ids_in_database)

In [179]:
df_full

,id,car_id,make,listing_price
0,147,a835a871-370a-4ce2-9250-26b64bb928ea,bmw,6250
1,148,2cfd860e-fa8e-4911-9544-6ec0fba9df1d,ford,6250
2,149,600f2c5e-dbb8-470a-ba56-558192b72a37,seat,6495
3,150,bd84208b-6533-4be6-aec8-cdd9d0c7b40f,citroen,1499
4,151,0d92d6f1-39e5-4606-b239-1eafa1eb65f2,fiat,9500
...,...,...,...,...
213193,339294,b0a459e6-2c9d-469e-a19c-245f664ca7e5,bmw,12600
213194,339295,fd7c197c-839f-44a5-90dc-6deeb720e4d2,mercedes-benz,12950
213195,339296,e8a027f7-da04-476e-b088-de74b3e75fc3,mercedes-benz,12693
213196,339297,c331e85f-9633-430b-9d65-3bd87a48a470,bmw,12950


In [175]:
id_to_remove = df_full.loc[(df_full.duplicated(subset=['car_id'],keep="first")), 'id'].values

In [176]:
id_to_remove

array([339332, 339335, 339336, ..., 341828, 341829, 341830], shape=(1582,))